![](https://drive.google.com/uc?export=view&id=13BVjDprd8KT7jLv3SmSBYTQjiCG6QqkC)
## **Загрузка справочников из НСИ**

## Протокол SOAP

* мой токен   
`464f7bce-cba9-414c-bcf2-d4287cb15cbb`      

In [1]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

In [2]:
# токен и прочее
user_key = "464f7bce-cba9-414c-bcf2-d4287cb15cbb"
wsdl = "https://nsi.rosminzdrav.ru/wsdl/SOAP-server.v2.php?wsdl"
endpoint = "https://nsi.rosminzdrav.ru/wsdl/SOAP-server.v2.php"

In [3]:
# следующая строка отвечает за подавление предупреждения
requests.packages.urllib3.disable_warnings()

# проверка доступности сервера
response = requests.get(wsdl, verify=False)
if response.status_code == 200:
    print("Всё хорошо, подключаемся")
else:
    print("Ошибка запроса:", response.status_code)

Всё хорошо, подключаемся


### 1. Получение списка справочников

In [4]:
%%time

# запрос на получение списка справочников
# тело запроса на получение списка справочников
body = (
    """<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/" xmlns:urn="urn:service">
   <soapenv:Header/>
   <soapenv:Body>
      <urn:getRefbookList>
         <userKey1>"""
    + str(user_key)
    + """</userKey1>
      </urn:getRefbookList>
   </soapenv:Body>
</soapenv:Envelope>
"""
)

# запрос
body = body.encode("utf-8")
session = requests.session()
session.headers = {"Content-Type": "text/xml; charset=utf-8"}
session.headers.update({"Content-Length": str(len(body))})
response = session.post(url=endpoint, data=body, verify=False)

# print(response.text)

CPU times: total: 0 ns
Wall time: 1.85 s


In [5]:
# извлечение данных и преобразование в dataframe
xml_string = str(response.text)

root = ET.fromstring(xml_string)
values = []
for value in root.findall(".//value"):
    values.append(value.text)

split_size = 5  # постоянное значение
splitted_list = [values[i : i + split_size] for i in range(0, len(values), split_size)]

df_reference = pd.DataFrame(splitted_list, columns=["1", "2", "3", "4", "5"])

In [6]:
# список справочников
df_reference

,1,2,3,4,5
0,48872,Справочник органов записи актов гражданского с...,<strong>АББРЕВИАТУРЫ:</strong>\n\n<ul>\n\t<li>...,1.2.643.5.1.13.13.99.2.832,L92K1B2BP5
1,20506,ФРМО. Справочник отделений и кабинетов,"Справочник отделений и кабинетов, содержит дан...",1.2.643.5.1.13.13.99.2.115,EP3FVD4DUL
2,19725,ФРМО. Справочник структурных подразделений,"Справочник структурных подразделений, содержит...",1.2.643.5.1.13.13.99.2.114,BR9AP1S2QB
3,43180,Реестр медицинских организаций Российской Феде...,Регистр медицинских организаций Российской Фед...,1.2.643.5.1.13.13.11.1461,MDR308
4,29780,Справочник регистрационных удостоверений и мод...,None,1.2.643.5.1.13.13.99.2.538,SF8U7PKK0U
...,...,...,...,...,...
1712,3063,Классификатор единиц измерения,Данному справочнику присвоен статус Архив.\n\n...,1.2.643.5.1.13.13.99.2.378,C20006
1713,3060,ФРБТ. Лекарственные формы лекарственных препар...,"Справочник носит локальный характер, разработа...",1.2.643.5.1.13.13.99.2.165,Q3B3AVPFSG
1714,2842,Номенклатура медицинских услуг,Справочник предназначен для унификации отчетно...,1.2.643.5.1.13.13.11.1070,5P7QT9V2NB
1715,2681,Характеристики высвобождения активных веществ ...,АББРЕВИАТУРЫ:\n\nЛП - Лекарственные препараты\...,1.2.643.5.1.13.13.99.2.798,9FD6PL3FO5


#### 1.1. Выбор справочника для работы

In [7]:
# oid справочника для поиска
oid = "1.2.643.5.1.13.13.11.1461"
df_reference[df_reference["4"] == oid]

,1,2,3,4,5
3,43180,Реестр медицинских организаций Российской Феде...,Регистр медицинских организаций Российской Фед...,1.2.643.5.1.13.13.11.1461,MDR308


In [8]:
index = df_reference[df_reference["4"] == oid].index[0]
# значение кода справочника для тестирования
reference_code = df_reference.iloc[index]["4"]
# название справочника
reference_name = df_reference.iloc[index]["2"]

### 2. Получение версии справочника

In [9]:
%%time

# тело запроса о получении  версии
body = (
    """<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/" xmlns:urn="urn:service">
   <soapenv:Header/>
   <soapenv:Body>
      <urn:getVersionList>
         <userKey5>"""
    + str(user_key)
    + """</userKey5>
         <refbookCode4>"""
    + str(reference_code)
    + """</refbookCode4>
      </urn:getVersionList>
   </soapenv:Body>
</soapenv:Envelope>
"""
)

# получение списка вресий справочника с кодом reference_code

body = body.encode("utf-8")
session = requests.session()
session.headers = {"Content-Type": "text/xml; charset=utf-8"}
session.headers.update({"Content-Length": str(len(body))})
response_ver = session.post(url=endpoint, data=body, verify=False)

CPU times: total: 15.6 ms
Wall time: 1.37 s


In [10]:
# извлечение данных и преобразование в dataframe
xml_string = str(response_ver.text)

root = ET.fromstring(xml_string)
values_ver = []
for value in root.findall(".//value"):
    values_ver.append(value.text)

split_size = 2
splitted_list = [
    values_ver[i : i + split_size] for i in range(0, len(values_ver), split_size)
]

df_version = pd.DataFrame(splitted_list, columns=["version", "version_datetime"])
df_version.insert(0, "reference", reference_code)

In [11]:
# преобразование даты справочника в формат datetime64[ns] и сортировка по возрастанию даты
df_version["version_datetime"] = pd.to_datetime(
    df_version["version_datetime"], format="%d.%m.%Y %H:%M"
)
df_version = df_version.sort_values(["version_datetime"]).reset_index(drop=True)
# последняя версия та, код которой соответствует максимальной дате
last_ver_code = df_version.iloc[(df_version.shape[0] - 1)]["version"]
last_ver_date = df_version.iloc[(df_version.shape[0] - 1)]["version_datetime"]
# код актуальной версии справочника
print(
    f"Актуальная версия справочника '{reference_name}' ({reference_code}): {last_ver_code}, дата версии: {last_ver_date}"
)

Актуальная версия справочника 'Реестр медицинских организаций Российской Федерации' (1.2.643.5.1.13.13.11.1461): 6.1129, дата версии: 2023-10-29 00:13:00


## c этого места можно переходить к получению записей справочника по протоколу REST
***

### 3. Получение структуры справочника

In [12]:
%%time

# получение структуры справочника
# структура в настоящее время нужна исключительно для простого получения количества столбцов
# тело запроса на получение структуры справочника

body = """<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/" xmlns:urn="urn:service">
   <soapenv:Header/>
   <soapenv:Body>
      <urn:getRefbookStructure>
         <userKey7>"""+str(user_key)+"""</userKey7>
         <refbookCode6>"""+str(reference_code)+"""</refbookCode6>
      </urn:getRefbookStructure>
   </soapenv:Body>
</soapenv:Envelope>"""

# получение структуры справочника с кодом reference_code

body = body.encode('utf-8')
session = requests.session()
session.headers = {"Content-Type": "text/xml; charset=utf-8"}
session.headers.update({"Content-Length": str(len(body))})
response_structure = session.post(url=endpoint, data=body, verify=False)
# print(response_structure.text)

CPU times: total: 0 ns
Wall time: 461 ms


In [13]:
# извлечение данных и формирование списков заголовков справочника
xml_string = str(response_structure.text)

root = ET.fromstring(xml_string)
values_structure = []
for value in root.findall(".//value"):
    values_structure.append(value.text)
# print(values_structure)

In [14]:
# заголовки
title_list = [values_structure[i] for i in range(1, len(values_structure), 5)]
# названия столбцов на русском языке
name_list = [values_structure[i] for i in range(0, len(values_structure), 5)]
# преобразование в dataframe
df_titles = pd.DataFrame({"title": title_list, "name": name_list})
if df_titles.empty:
    print("Список пуст, что-то пошло не так...")
else:
    print("Список заголовков успешно сформирован")

Список заголовков успешно сформирован


In [15]:
# полное описание полей таблицы
split_size = 5
title_description = pd.DataFrame(
    [
        values_structure[i : i + split_size]
        for i in range(0, len(values_structure), split_size)
    ],
    columns=["name_rus", "name_eng", "type", "size", "nothin"],
)

In [16]:
title_description

,name_rus,name_eng,type,size,nothin
0,Дата создания записи,createDate,DATE,None,None
1,Полное наименование,nameFull,VARCHAR2,1000 byte,None
2,Код региона,addrRegionId,NUMBER,None,None
3,"OID СП-1, дочерних для организации",childrenOspOid,VARCHAR2,1000 byte,None
4,Уровень организации,moLevel,VARCHAR2,1000 byte,None
5,Наименование населенного пункта,areaName,VARCHAR2,1000 byte,None
6,OID СП-1,ospOid,VARCHAR2,1000 byte,None
7,Наименование субъекта системы здравоохранения,medicalSubjectName,VARCHAR2,1000 byte,None
8,Наименование улицы,streetName,VARCHAR2,1000 byte,None
9,Долгота,longtitude,VARCHAR2,1000 byte,None


In [17]:
# сохранение
title_description.to_csv(
    "C:/work/scripts/описания_полей/description.csv",
    sep=";",
    encoding="utf-8-sig",
    index=False,
)

### 4. Получение записей справочника

In [18]:
%%time

# запрос на получение записей справочника
# тело запроса на прлучение версии

body = """<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/" xmlns:urn="urn:service">
   <soapenv:Header/>
   <soapenv:Body>
      <urn:getRefbook>
         <userKey>"""+str(user_key)+"""</userKey>
         <refbookCode>"""+str(reference_code)+"""</refbookCode>
         <version>"""+str(last_ver_code)+"""</version>
      </urn:getRefbook>
   </soapenv:Body>
</soapenv:Envelope></soapenv:Envelope>
"""

# получение списка записей справочника с кодом reference_code и версией last_ver_code

body = body.encode('utf-8')
session = requests.session()
session.headers = {"Content-Type": "text/xml; charset=utf-8"}
session.headers.update({"Content-Length": str(len(body))})
response_entry = session.post(url=endpoint, data=body, verify=False)

print(response_entry.text)

<soap:Envelope xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/"><soap:Body><ns1:getRefbookResponse xmlns:ns1="urn:service"><getRefbookReturn><item><children><item><children/><key>03x0003</key><value>Справочник слишком большой</value></item></children></item></getRefbookReturn></ns1:getRefbookResponse></soap:Body></soap:Envelope>
CPU times: total: 0 ns
Wall time: 1.09 s


In [18]:
# извлечение данных и преобразование в dataframe
xml_string = str(response_entry.text)

root = ET.fromstring(xml_string)
values_ver = []
for value in root.findall(".//value"):
    values_ver.append(value.text)

# извлечение структуры и преобразование в dataframe

root = ET.fromstring(xml_string)
headers_ver = []
for key in root.findall(".//key"):
    headers_ver.append(key.text)

# извлечение данных и формирование справочника
split_size = len(title_list)
splitted_list = [
    values_ver[i : i + split_size] for i in range(0, len(values_ver), split_size)
]
title_list_short = headers_ver[0:split_size]

df_ver = pd.DataFrame(splitted_list, columns=title_list_short)

# формирование описания полей
df_title_list_short = pd.DataFrame({"title_r": title_list_short})
df_title_list_short = df_title_list_short.merge(
    df_titles, left_on="title_r", right_on="title", how="left"
)[["title", "name"]]
print(f"\n\tОписание {df_title_list_short.shape[0]} полей справочника\n")
print(df_title_list_short.to_string(index=True))


	Описание 1 полей справочника

  title name
0   NaN  NaN


In [19]:
df_ver

,03x0003
0,Справочник слишком большой


### 4. Получение количества частей записей версии справочника   
_Возвращается правильный, но пустой ответ_

In [22]:
str(last_ver_code)

'2.1330'

In [23]:
%%time

# получение количества частей записи
# тело запроса

body = """<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/" xmlns:urn="urn:service">
   <soapenv:Header/>
   <soapenv:Body>
      <urn:getRefbookParts>
         <userKey>"""+str(user_key)+"""</userKey>
         <refbookCode>"""+str(reference_code)+"""</refbookCode>
         <version>"""+str(last_ver_code)+"""</version>
      </urn:getRefbookParts>
   </soapenv:Body>
</soapenv:Envelope>"""

# получение количества частей записи справочника с кодом reference_code и версией last_ver_code

body = body.encode('utf-8')
session = requests.session()
session.headers = {"Content-Type": "text/xml; charset=utf-8"}
session.headers.update({"Content-Length": str(len(body))})
response_entry = session.post(url=endpoint, data=body, verify=False)

print(response_entry.text)


CPU times: total: 0 ns
Wall time: 330 ms


In [24]:
# извлечение данных и преобразование в dataframe
xml_string = str(response_entry.text)

root = ET.fromstring(xml_string)
values_ver = []
for value in root.findall(".//value"):
    values_ver.append(value.text)

ParseError: no element found: line 1, column 0 (<string>)

### 5. Получение части записей версии
_Возвращается правильный, но пустой ответ_

In [21]:
# номер части для тестирования
part_number = 1

In [22]:
%%time

# получение части записи
# тело запроса

body = """<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/" xmlns:urn="urn:service">
   <soapenv:Header/>
   <soapenv:Body>
      <urn:getRefbookPartial>
         <userKey>"""+str(user_key)+"""</userKey>
         <refbookCode>"""+str(reference_code)+"""</refbookCode>
         <version>"""+str(last_ver_code)+"""</version>
         <partNumber>"""+str(part_number)+"""</partNumber>
      </urn:getRefbookPartial>
   </soapenv:Body>
</soapenv:Envelope>"""

# получение part_number записи справочника с кодом reference_code и версией last_ver_code

body = body.encode('utf-8')
session = requests.session()
session.headers = {"Content-Type": "text/xml; charset=utf-8"}
session.headers.update({"Content-Length": str(len(body))})
response_entry = session.post(url=endpoint, data=body, verify=False)

print(response_entry.text)


CPU times: total: 15.6 ms
Wall time: 455 ms


## REST-СЕРВИС

In [19]:
import requests

url = "https://nsi.rosminzdrav.ru/port/rest/searchDictionary"
params = {
    "userKey": "464f7bce-cba9-414c-bcf2-d4287cb15cbb",
    "identifier": "1.2.643.5.1.13.13.99.2.533",
    "page": "1",
    "size": "2",
}
headers = {
    "accept": "application/json;charset=UTF-8",
    "content-type": "application/json",
}

response = requests.get(url, params=params, headers=headers, verify=False)

if response.status_code == 200:
    print("Всё хорошо, подключаемся")
    data = response.json()
    # обработка полученных данных
else:
    print("Ошибка запроса:", response.status_code)

Всё хорошо, подключаемся


In [20]:
data

{'result': 'OK',
 'resultCode': None,
 'resultText': None,
 'total': 1,
 'list': [{'result': None,
   'resultText': None,
   'resultCode': None,
   'identifier': '3HK3BU6AIE',
   'oid': '1.2.643.5.1.13.13.99.2.533',
   'version': '3.36',
   'rowsCount': 451,
   'createDate': '17.12.2019 09:58',
   'publishDate': '06.10.2023 12:53',
   'lastUpdate': '06.10.2023 12:53',
   'fullName': 'Нормативно-правовые акты справочников ФР НСИ',
   'shortName': 'НПА справочников ФР НСИ',
   'description': '<strong>АББРЕВИАТУРЫ: </strong>\n\nНПА - Нормативно-правовой акт\n\n<strong>НАЗНАЧЕНИЕ:</strong>\n\nСправочник содержит перечень нормативно-правовых актов, которые используются для наполнения паспортов справочников, представленных на портале НСИ.\n\n<strong>ДОПОЛНИТЕЛЬНЫЕ ИСТОЧНИКИ ИНФОРМАЦИИ:</strong>\n\nОтсутствуют',
   'structureNotes': '1)ID, Код, уникальный код записи, целое число; обязательное поле;\n2)TYPENPA, Наименование вида НПА, текстовый формат, обязательное поле;\n3)DATEOFADOPTION, Дата

In [21]:
train = pd.DataFrame.from_dict(data, orient="index").reset_index()

In [33]:
train

,index,0
0,result,OK
1,resultCode,None
2,resultText,None
3,total,1
4,list,"[{'result': None, 'resultText': None, 'resultC..."


In [22]:
df_list = pd.DataFrame.from_dict(train.iloc[4][0][0], orient="index")

In [23]:
df_list

,0
result,None
resultText,None
resultCode,None
identifier,3HK3BU6AIE
oid,1.2.643.5.1.13.13.99.2.533
version,3.36
rowsCount,451
createDate,17.12.2019 09:58
publishDate,06.10.2023 12:53
lastUpdate,06.10.2023 12:53


***
***
### с этого места нужно получать записи выбранной версии выбранного справочника после работы по протколу SOAP

In [24]:
# сначала убедиться, что все данные есть
print(
    f"Актуальная версия справочника '{reference_name}' ({reference_code}): {last_ver_code}, дата версии: {last_ver_date}"
)

Актуальная версия справочника 'Реестр медицинских организаций Российской Федерации' (1.2.643.5.1.13.13.11.1461): 6.1129, дата версии: 2023-10-29 00:13:00


### Получение записей версии справочника  

In [25]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

# следующая строка отвечает за подавление предупреждения
requests.packages.urllib3.disable_warnings()

In [26]:
userKey = "464f7bce-cba9-414c-bcf2-d4287cb15cbb"
identifier = reference_code
version = last_ver_code
# не совсем понятное
page = "1"
# количество записей в выгрузке
size = "100000"

In [27]:
%%time

url = "https://nsi.rosminzdrav.ru/port/rest/data?"
params = {
    "userKey": userKey,
    "identifier": identifier,
    "version": version,
    "page": page,
    "size": size,
}
headers = {
    "accept": "application/json;charset=UTF-8",
    "content-type": "application/json",
}

response = requests.get(url, params=params, headers=headers, verify=False)

if response.status_code == 200:
    print("Всё хорошо, подключаемся")
    data = response.json()
    # обработка полученных данных
else:
    print("Ошибка запроса:", response.status_code)

KeyboardInterrupt: 

In [24]:
data

{'result': 'OK',
 'resultText': None,
 'resultCode': None,
 'total': 75961,
 'list': [[{'column': 'id', 'value': '7043902'},
   {'column': 'oid', 'value': '1.2.643.5.1.13.13.12.2.72.89181'},
   {'column': 'oldOid', 'value': None},
   {'column': 'nameFull',
    'value': 'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ПАЛОМА"'},
   {'column': 'nameShort', 'value': 'ООО "ПАЛОМА"'},
   {'column': 'parentId', 'value': None},
   {'column': 'medicalSubjectId', 'value': '2'},
   {'column': 'medicalSubjectName', 'value': 'Медицинская организация'},
   {'column': 'inn', 'value': '7204167956'},
   {'column': 'kpp', 'value': '720301001'},
   {'column': 'ogrn', 'value': '1117232016362'},
   {'column': 'regionId', 'value': '72'},
   {'column': 'regionName', 'value': 'Тюменская область'},
   {'column': 'organizationType', 'value': '2'},
   {'column': 'moDeptId', 'value': '0'},
   {'column': 'moDeptName', 'value': None},
   {'column': 'founder', 'value': 'САРЫЧЕВА МАРИНА НИКОЛАЕВНА'},
   {'column': 'delete

In [29]:
%%time

# пустой df с заголовками
columns_list = pd.DataFrame(data["list"][0]).T.iloc[0].to_list()
df_ref = pd.DataFrame(columns=columns_list)

data_list = data["list"]
for element in data_list:
    df_row = pd.DataFrame(element).reset_index(drop=True).T
    df_row = df_row.set_axis(columns_list, axis="columns")
    df_row = df_row.drop(index="column").reset_index(drop=True)
    df_ref = pd.concat([df_ref, df_row])
df_ref = df_ref.reset_index(drop=True)
df_ref = df_ref.drop_duplicates()

CPU times: total: 18min 27s
Wall time: 53min 41s


In [30]:
df_ref

,id,oid,oldOid,nameFull,nameShort,parentId,medicalSubjectId,medicalSubjectName,inn,kpp,...,prefixStreet,house,building,struct,parentOspOid,ospOid,childrenOspOid,deleteDate,createDate,modifyDate
0,7043902,1.2.643.5.1.13.13.12.2.72.89181,None,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ПАЛОМА""","ООО ""ПАЛОМА""",None,2,Медицинская организация,7204167956,720301001,...,ул,34,None,None,None,None,None,None,20.07.2023,20.07.2023
1,7043901,1.2.643.5.1.13.13.12.2.78.89180,None,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""САНК...","ООО""ЦДИ""",None,2,Медицинская организация,7813452101,781301001,...,пр-кт,9/1,Литера А,None,None,None,None,None,20.07.2023,20.07.2023
2,7043899,1.2.643.5.1.13.13.12.3.27.89177,None,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ВОДО...","ООО ""ВОДОЛЕЙ +""",None,3,Фармацевтическая организация,2709012708,270901001,...,ул,11,None,None,None,None,None,None,20.07.2023,20.07.2023
3,7043891,1.2.643.5.1.13.13.12.2.77.89167,None,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЗАФФ...","ООО ""ЗАФФИРО КЛИНИК""",None,2,Медицинская организация,7707431340,770701001,...,None,None,None,None,None,None,None,None,20.07.2023,20.07.2023
4,7043889,1.2.643.5.1.13.13.12.2.72.89165,None,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ПРИО...","ООО ""ПРИОРИТИ""",None,2,Медицинская организация,7203515520,720301001,...,None,None,None,None,None,None,None,None,20.07.2023,20.07.2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75956,1732,1.2.643.5.1.13.13.12.2.73.7370,None,ГОСУДАРСТВЕННОЕ УЧРЕЖДЕНИЕ ЗДРАВООХРАНЕНИЯ УЛЬ...,ГУЗ УОКБ,None,2,Медицинская организация,7300000043,732601001,...,None,7,None,None,None,None,None,None,14.08.2022,14.03.2023
75957,1730,1.2.643.5.1.13.13.12.2.73.7369,None,"ГОСУДАРСТВЕННОЕ УЧРЕЖДЕНИЕ ЗДРАВООХРАНЕНИЯ ""УЛ...","ГУЗ ""МИАЦ""",None,2,Медицинская организация,7325031357,732501001,...,None,None,None,None,None,None,None,None,14.08.2022,18.04.2023
75958,1724,1.2.643.5.1.13.13.12.2.77.7798,1.2.643.5.1.13.3.25.77.31,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ НАУЧНОЕ ...,"ФГБНУ ""НИИ МТ""",None,2,Медицинская организация,7719022912,771901001,...,None,12,None,None,None,None,None,None,14.08.2022,01.03.2023
75959,1683,1.2.643.5.1.13.13.12.2.22.1748,None,КРАЕВОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ З...,"КГБУЗ ""АКОД""",None,2,Медицинская организация,2225022328,222501001,...,None,110к,None,None,None,None,None,None,14.08.2022,26.05.2023
